In [125]:
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import ScalarFormatter
from astropy.utils.data import download_file
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
from scipy.stats import linregress
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib as mpl
from scipy import stats
import pandas as pd
import logging, sys
import numpy as np
import statistics
import warnings
import argparse
import datetime
import logging
import shutil
import time
import glob
import math
import sys
import csv
import os

#Author: Angelica Psaridi

In [138]:

warnings.filterwarnings("ignore")

logging.disable(sys.maxsize)
if not sys.warnoptions:
    warnings.simplefilter("ignore")

logging.disable(sys.maxsize)




In [139]:
data_dir_base = "/Users/angelicapsaridi/Documents/Geneva/Recommissioning/Data/"
path =  "/Users/angelicapsaridi/Documents/Geneva/Recommissioning/Tables/"


In [143]:
date_list = []
date = []
date_eve = []
date_mor = []



os.chdir(data_dir_base)

for folder in glob.glob('2022*'):
    date_list.append(folder)
    date.append(folder)
    date_eve.append(folder)
    date_mor.append(folder)


date_list.sort()
date_eve.sort()
date_mor.sort()
date.sort()




In [144]:
bias_eve1 = []                    
bias_all_eve = []
bias_mor1 = []                    
bias_all_mor = []
                  




for i in range(len(date_list)):
    os.chdir(data_dir_base+date_list[i])
    bias_all_eve.append([])
    bias_all_mor.append([])
    for frame in glob.glob("ECAM.*.fits"):
        hdul = fits.open(frame)
        hdr = hdul[0].header 
        if hdr['HIERARCH OGE OBS TYPE'] == "BIAS":
            if hdr['HIERARCH OGE DET OUT RNAME'] == "ALL":
                hour = hdr['HIERARCH OGE OBS HOURSTU'][0:2]
                if hour=="19" or hour=="20" or hour=="21" or hour=="22" or hour=="23" or hour=="00" or hour=="01" or hour=="02":
                    bias_all_eve[-1].append(hdr['ARCFILE'])
                elif hour=="06" or hour=="07" or hour=="08" or hour=="09" or hour=="10" or hour=="11" or hour=="13" or hour=="13":
                    bias_all_mor[-1].append(hdr['ARCFILE'])


                    



In [146]:
bias_eve = []                    
bias_mor = []  


for i in range(len(date_list)):
    if (len(bias_all_eve[i])<=3):
        date_eve.remove(date_list[i]) 
    else:
        bias_eve1.append(bias_all_eve[i])
        
for i in range(len(date_list)):
    if (len(bias_all_mor[i])<=3):
        date_mor.remove(date_list[i]) 
    else:
        bias_mor1.append(bias_all_mor[i])

In [147]:
for i in range(len(date_eve)):
    if (len(bias_eve1[i])>=5):
        bias_eve.append(bias_eve1[i][0:5])
    else:
        bias_eve.append(bias_eve1[i])


for i in range(len(date_mor)):
    if (len(bias_mor1[i])>=5):
        bias_mor.append(bias_mor1[i][0:5])
    else:
        bias_mor.append(bias_mor1[i])


In [148]:
for i in range(len(date_eve)):
    print(date_eve[i],bias_eve[i])

2022-02-16 ['ECAM.2022-02-16T23:24:11.000.fits', 'ECAM.2022-02-16T23:24:22.000.fits', 'ECAM.2022-02-16T23:24:00.000.fits', 'ECAM.2022-02-16T23:23:49.000.fits', 'ECAM.2022-02-16T23:23:38.000.fits']
2022-05-15 ['ECAM.2022-05-15T21:51:27.000.fits', 'ECAM.2022-05-15T21:53:06.000.fits', 'ECAM.2022-05-15T21:52:00.000.fits', 'ECAM.2022-05-15T21:51:38.000.fits', 'ECAM.2022-05-15T21:52:55.000.fits']
2022-06-01 ['ECAM.2022-06-02T01:32:16.000.fits', 'ECAM.2022-06-02T01:34:07.000.fits', 'ECAM.2022-06-01T21:40:11.000.fits', 'ECAM.2022-06-02T01:33:56.000.fits', 'ECAM.2022-06-02T01:35:13.000.fits']
2022-07-11 ['ECAM.2022-07-11T22:26:30.000.fits', 'ECAM.2022-07-11T22:25:57.000.fits', 'ECAM.2022-07-11T22:26:19.000.fits', 'ECAM.2022-07-11T22:26:41.000.fits', 'ECAM.2022-07-11T22:26:08.000.fits']
2022-09-04 ['ECAM.2022-09-04T21:25:45.000.fits', 'ECAM.2022-09-04T21:25:34.000.fits', 'ECAM.2022-09-04T21:25:01.000.fits', 'ECAM.2022-09-04T21:25:23.000.fits', 'ECAM.2022-09-04T21:26:08.000.fits']
2022-09-06 ['EC

In [149]:
import pandas as pd


def analysis_bias(a):
    return np.mean(a),np.std(a)

def filter_bias(a,mean,std,sigmas):
    distance_from_mean = abs(a - mean)
    not_outlier = distance_from_mean < sigmas * std
    no_outliers = a[not_outlier]
    
    return no_outliers

def convert_1D(a):
    a_1D=[]
    for i in range(len(a)):
        for j in range(len(a[i])):
            a_1D.append(a[i][j])
            
    return a_1D
            

In [151]:
#Evening

bias_LR=[]
mean_bias_LR=[]
std_bias_LR=[]
outliers_LR=[]
filtered_bias_LR=[]
mean_bias_LR_night_eve=[]
mean_std_LR_night_eve=[]
temp=[]
mean_temp_eve=[]
rnoise_LR=[]
mean_rnoise_LR_eve=[]
std_rnoise_LR_eve=[]
mean_rnoise_LR_eve_night=[]
std_rnoise_LR_eve_night=[]


for i in range(len(date_eve)):
    print('Date analyzed now:',date_eve[i])
    os.chdir(data_dir_base+date_eve[i])

#Lower right quadrant
    mean_bias_LR.append([])
    std_bias_LR.append([])
    outliers_LR.append([])
    filtered_bias_LR.append([])
    temp.append([])
    rnoise_LR.append([])
    bias_LR.append([])
    for k in range(len(bias_eve[i])):
        print(int((k+1)/len(bias_eve[i])*100),'%')
        LR = fits.open(bias_eve[i][k])
        temp[-1].append(LR[0].header['HIERARCH OGE AMBI TEMP'])
        LR[0].data = LR[0].data[101:2156,2149:4196]
        bias_data = LR[0].data
        bias_LR[-1].append(bias_data)
        mean,std=analysis_bias(bias_data)
        bias_filtered = filter_bias(bias_data,mean,std,5.)  
        filtered_bias_LR[-1].append(bias_filtered)
        mean_filtered,std_filtered=analysis_bias(bias_filtered)
        mean_bias_LR[-1].append(mean_filtered)
        std_bias_LR[-1].append(std_filtered)
    mean_bias_LR_night_eve.append(np.mean(mean_bias_LR[-1]))        
    mean_std_LR_night_eve.append(np.mean(std_bias_LR[-1]))  
    mean_temp_eve.append(np.mean(temp[-1]))
    for j in range(len(bias_LR[-1])):
        os.chdir(data_dir_base+date[i])
        print(int((j+1)/len(bias_LR[-1])*100),'%')
        for k in range(j+1):
                if j!=k:
                    a = np.array(bias_LR[-1][j],dtype=np.int16)
                    b = np.array(bias_LR[-1][k],dtype=np.int16)
                    diff_bias = np.abs(np.subtract(a,b))
                    mean,std=analysis_bias(diff_bias)
                    diff_bias_filtered = filter_bias(diff_bias,mean,std,5.)
                    rnoise_LR[-1].append(np.std(diff_bias_filtered)/math.sqrt(2.))
                
    mean_rnoise_LR_eve.append(np.mean(rnoise_LR[-1]))     
    std_rnoise_LR_eve.append(np.std(rnoise_LR[-1])) 


bias_LL=[]
mean_bias_LL=[]
std_bias_LL=[]
outliers_LL=[]
filtered_bias_LL=[]
mean_bias_LL_night_eve=[]
mean_std_LL_night_eve=[]
temp=[]
mean_temp_eve=[]
rnoise_LL=[]
mean_rnoise_LL_eve=[]
std_rnoise_LL_eve=[]
mean_rnoise_LL_eve_night=[]
std_rnoise_LL_eve_night=[]


for i in range(len(date_eve)):
    print('Date analyzed now:',date_eve[i])
    os.chdir(data_dir_base+date_eve[i])

#Lower right quadrant
    mean_bias_LL.append([])
    std_bias_LL.append([])
    outliers_LL.append([])
    filtered_bias_LL.append([])
    temp.append([])
    rnoise_LL.append([])
    bias_LL.append([])
    for k in range(len(bias_eve[i])):
        print(int((k+1)/len(bias_eve[i])*100),'%')
        LL = fits.open(bias_eve[i][k])
        temp[-1].append(LL[0].header['HIERARCH OGE AMBI TEMP'])
        LL[0].data = LL[0].data[101:2156,101:2148]
        bias_data = LL[0].data
        bias_LL[-1].append(bias_data)
        mean,std=analysis_bias(bias_data)
        bias_filtered = filter_bias(bias_data,mean,std,5.)  
        filtered_bias_LL[-1].append(bias_filtered)
        mean_filtered,std_filtered=analysis_bias(bias_filtered)
        mean_bias_LL[-1].append(mean_filtered)
        std_bias_LL[-1].append(std_filtered)
    mean_bias_LL_night_eve.append(np.mean(mean_bias_LL[-1]))        
    mean_std_LL_night_eve.append(np.mean(std_bias_LL[-1]))  
    mean_temp_eve.append(np.mean(temp[-1]))
    for j in range(len(bias_LL[-1])):
        os.chdir(data_dir_base+date[i])
        print(int((j+1)/len(bias_LL[-1])*100),'%')
        for k in range(j+1):
                if j!=k:
                    a = np.array(bias_LL[-1][j],dtype=np.int16)
                    b = np.array(bias_LL[-1][k],dtype=np.int16)
                    diff_bias = np.abs(np.subtract(a,b))
                    mean,std=analysis_bias(diff_bias)
                    diff_bias_filtered = filter_bias(diff_bias,mean,std,5.)
                    rnoise_LL[-1].append(np.std(diff_bias_filtered)/math.sqrt(2.))
                
    mean_rnoise_LL_eve.append(np.mean(rnoise_LL[-1]))     
    std_rnoise_LL_eve.append(np.std(rnoise_LL[-1])) 



bias_UL=[]
mean_bias_UL=[]
std_bias_UL=[]
outliers_UL=[]
filtered_bias_UL=[]
mean_bias_UL_night_eve=[]
mean_std_UL_night_eve=[]
temp=[]
mean_temp_eve=[]
rnoise_UL=[]
mean_rnoise_UL_eve=[]
std_rnoise_UL_eve=[]
mean_rnoise_UL_eve_night=[]
std_rnoise_UL_eve_night=[]


for i in range(len(date_eve)):
    print('Date analyzed now:',date_eve[i])
    os.chdir(data_dir_base+date_eve[i])

#Lower right quadrant
    mean_bias_UL.append([])
    std_bias_UL.append([])
    outliers_UL.append([])
    filtered_bias_UL.append([])
    temp.append([])
    rnoise_UL.append([])
    bias_UL.append([])
    for k in range(len(bias_eve[i])):
        print(int((k+1)/len(bias_eve[i])*100),'%')
        UL = fits.open(bias_eve[i][k])
        temp[-1].append(UL[0].header['HIERARCH OGE AMBI TEMP'])
        UL[0].data = UL[0].data[2157:4212,101:2148]
        bias_data = UL[0].data
        bias_UL[-1].append(bias_data)
        mean,std=analysis_bias(bias_data)
        bias_filtered = filter_bias(bias_data,mean,std,5.)  
        filtered_bias_UL[-1].append(bias_filtered)
        mean_filtered,std_filtered=analysis_bias(bias_filtered)
        mean_bias_UL[-1].append(mean_filtered)
        std_bias_UL[-1].append(std_filtered)
    mean_bias_UL_night_eve.append(np.mean(mean_bias_UL[-1]))        
    mean_std_UL_night_eve.append(np.mean(std_bias_UL[-1]))  
    mean_temp_eve.append(np.mean(temp[-1]))
    for j in range(len(bias_UL[-1])):
        os.chdir(data_dir_base+date[i])
        print(int((j+1)/len(bias_UL[-1])*100),'%')
        for k in range(j+1):
                if j!=k:
                    a = np.array(bias_UL[-1][j],dtype=np.int16)
                    b = np.array(bias_UL[-1][k],dtype=np.int16)
                    diff_bias = np.abs(np.subtract(a,b))
                    mean,std=analysis_bias(diff_bias)
                    diff_bias_filtered = filter_bias(diff_bias,mean,std,5.)
                    rnoise_UL[-1].append(np.std(diff_bias_filtered)/math.sqrt(2.))
                
    mean_rnoise_UL_eve.append(np.mean(rnoise_UL[-1]))     
    std_rnoise_UL_eve.append(np.std(rnoise_UL[-1])) 

bias_UR=[]
mean_bias_UR=[]
std_bias_UR=[]
outliers_UR=[]
filtered_bias_UR=[]
mean_bias_UR_night_eve=[]
mean_std_UR_night_eve=[]
temp=[]
mean_temp_eve=[]
rnoise_UR=[]
mean_rnoise_UR_eve=[]
std_rnoise_UR_eve=[]
mean_rnoise_UR_eve_night=[]
std_rnoise_UR_eve_night=[]


for i in range(len(date_eve)):
    print('Date analyzed now:',date_eve[i])
    os.chdir(data_dir_base+date_eve[i])

#Lower right quadrant
    mean_bias_UR.append([])
    std_bias_UR.append([])
    outliers_UR.append([])
    filtered_bias_UR.append([])
    temp.append([])
    rnoise_UR.append([])
    bias_UR.append([])
    for k in range(len(bias_eve[i])):
        print(int((k+1)/len(bias_eve[i])*100),'%')
        UR = fits.open(bias_eve[i][k])
        temp[-1].append(UR[0].header['HIERARCH OGE AMBI TEMP'])
        UR[0].data = UR[0].data[2157:4212,2149:4196]
        bias_data = UR[0].data
        bias_UR[-1].append(bias_data)
        mean,std=analysis_bias(bias_data)
        bias_filtered = filter_bias(bias_data,mean,std,5.)  
        filtered_bias_UR[-1].append(bias_filtered)
        mean_filtered,std_filtered=analysis_bias(bias_filtered)
        mean_bias_UR[-1].append(mean_filtered)
        std_bias_UR[-1].append(std_filtered)
    mean_bias_UR_night_eve.append(np.mean(mean_bias_UR[-1]))        
    mean_std_UR_night_eve.append(np.mean(std_bias_UR[-1]))  
    mean_temp_eve.append(np.mean(temp[-1]))
    for j in range(len(bias_UR[-1])):
        os.chdir(data_dir_base+date[i])
        print(int((j+1)/len(bias_UR[-1])*100),'%')
        for k in range(j+1):
                if j!=k:
                    a = np.array(bias_UR[-1][j],dtype=np.int16)
                    b = np.array(bias_UR[-1][k],dtype=np.int16)
                    diff_bias = np.abs(np.subtract(a,b))
                    mean,std=analysis_bias(diff_bias)
                    diff_bias_filtered = filter_bias(diff_bias,mean,std,5.)
                    rnoise_UR[-1].append(np.std(diff_bias_filtered)/math.sqrt(2.))
                
    mean_rnoise_UR_eve.append(np.mean(rnoise_UR[-1]))     
    std_rnoise_UR_eve.append(np.std(rnoise_UR[-1])) 




Date analyzed now: 2022-02-16
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-05-15
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-06-01
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-07-11
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-09-04
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-09-06
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-09-18
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-10-03
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-10-14
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-11-07
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-11-12
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-11-26
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed no

In [152]:
#morning

bias_LR=[]
mean_bias_LR=[]
std_bias_LR=[]
outliers_LR=[]
filtered_bias_LR=[]
mean_bias_LR_night_mor=[]
mean_std_LR_night_mor=[]
temp=[]
mean_temp_mor=[]
rnoise_LR=[]
mean_rnoise_LR_mor=[]
std_rnoise_LR_mor=[]
mean_rnoise_LR_mor_night=[]
std_rnoise_LR_mor_night=[]


for i in range(len(date_mor)):
    print('Date analyzed now:',date_mor[i])
    os.chdir(data_dir_base+date_mor[i])

#Lower right quadrant
    mean_bias_LR.append([])
    std_bias_LR.append([])
    outliers_LR.append([])
    filtered_bias_LR.append([])
    temp.append([])
    rnoise_LR.append([])
    bias_LR.append([])
    for k in range(len(bias_mor[i])):
        print(int((k+1)/len(bias_mor[i])*100),'%')
        LR = fits.open(bias_mor[i][k])
        temp[-1].append(LR[0].header['HIERARCH OGE AMBI TEMP'])
        LR[0].data = LR[0].data[101:2156,2149:4196]
        bias_data = LR[0].data
        bias_LR[-1].append(bias_data)
        mean,std=analysis_bias(bias_data)
        bias_filtered = filter_bias(bias_data,mean,std,5.)  
        filtered_bias_LR[-1].append(bias_filtered)
        mean_filtered,std_filtered=analysis_bias(bias_filtered)
        mean_bias_LR[-1].append(mean_filtered)
        std_bias_LR[-1].append(std_filtered)
    mean_bias_LR_night_mor.append(np.mean(mean_bias_LR[-1]))        
    mean_std_LR_night_mor.append(np.mean(std_bias_LR[-1]))  
    mean_temp_mor.append(np.mean(temp[-1]))
    for j in range(len(bias_LR[-1])):
        os.chdir(data_dir_base+date[i])
        print(int((j+1)/len(bias_LR[-1])*100),'%')
        for k in range(j+1):
                if j!=k:
                    a = np.array(bias_LR[-1][j],dtype=np.int16)
                    b = np.array(bias_LR[-1][k],dtype=np.int16)
                    diff_bias = np.abs(np.subtract(a,b))
                    mean,std=analysis_bias(diff_bias)
                    diff_bias_filtered = filter_bias(diff_bias,mean,std,5.)
                    rnoise_LR[-1].append(np.std(diff_bias_filtered)/math.sqrt(2.))
                
    mean_rnoise_LR_mor.append(np.mean(rnoise_LR[-1]))     
    std_rnoise_LR_mor.append(np.std(rnoise_LR[-1])) 


bias_LL=[]
mean_bias_LL=[]
std_bias_LL=[]
outliers_LL=[]
filtered_bias_LL=[]
mean_bias_LL_night_mor=[]
mean_std_LL_night_mor=[]
temp=[]
mean_temp_mor=[]
rnoise_LL=[]
mean_rnoise_LL_mor=[]
std_rnoise_LL_mor=[]
mean_rnoise_LL_mor_night=[]
std_rnoise_LL_mor_night=[]


for i in range(len(date_mor)):
    print('Date analyzed now:',date_mor[i])
    os.chdir(data_dir_base+date_mor[i])

#Lower right quadrant
    mean_bias_LL.append([])
    std_bias_LL.append([])
    outliers_LL.append([])
    filtered_bias_LL.append([])
    temp.append([])
    rnoise_LL.append([])
    bias_LL.append([])
    for k in range(len(bias_mor[i])):
        print(int((k+1)/len(bias_mor[i])*100),'%')
        LL = fits.open(bias_mor[i][k])
        temp[-1].append(LL[0].header['HIERARCH OGE AMBI TEMP'])
        LL[0].data = LL[0].data[101:2156,101:2148]
        bias_data = LL[0].data
        bias_LL[-1].append(bias_data)
        mean,std=analysis_bias(bias_data)
        bias_filtered = filter_bias(bias_data,mean,std,5.)  
        filtered_bias_LL[-1].append(bias_filtered)
        mean_filtered,std_filtered=analysis_bias(bias_filtered)
        mean_bias_LL[-1].append(mean_filtered)
        std_bias_LL[-1].append(std_filtered)
    mean_bias_LL_night_mor.append(np.mean(mean_bias_LL[-1]))        
    mean_std_LL_night_mor.append(np.mean(std_bias_LL[-1]))  
    mean_temp_mor.append(np.mean(temp[-1]))
    for j in range(len(bias_LL[-1])):
        os.chdir(data_dir_base+date[i])
        print(int((j+1)/len(bias_LL[-1])*100),'%')
        for k in range(j+1):
                if j!=k:
                    a = np.array(bias_LL[-1][j],dtype=np.int16)
                    b = np.array(bias_LL[-1][k],dtype=np.int16)
                    diff_bias = np.abs(np.subtract(a,b))
                    mean,std=analysis_bias(diff_bias)
                    diff_bias_filtered = filter_bias(diff_bias,mean,std,5.)
                    rnoise_LL[-1].append(np.std(diff_bias_filtered)/math.sqrt(2.))
                
    mean_rnoise_LL_mor.append(np.mean(rnoise_LL[-1]))     
    std_rnoise_LL_mor.append(np.std(rnoise_LL[-1])) 



bias_UL=[]
mean_bias_UL=[]
std_bias_UL=[]
outliers_UL=[]
filtered_bias_UL=[]
mean_bias_UL_night_mor=[]
mean_std_UL_night_mor=[]
temp=[]
mean_temp_mor=[]
rnoise_UL=[]
mean_rnoise_UL_mor=[]
std_rnoise_UL_mor=[]
mean_rnoise_UL_mor_night=[]
std_rnoise_UL_mor_night=[]


for i in range(len(date_mor)):
    print('Date analyzed now:',date_mor[i])
    os.chdir(data_dir_base+date_mor[i])

#Lower right quadrant
    mean_bias_UL.append([])
    std_bias_UL.append([])
    outliers_UL.append([])
    filtered_bias_UL.append([])
    temp.append([])
    rnoise_UL.append([])
    bias_UL.append([])
    for k in range(len(bias_mor[i])):
        print(int((k+1)/len(bias_mor[i])*100),'%')
        UL = fits.open(bias_mor[i][k])
        temp[-1].append(UL[0].header['HIERARCH OGE AMBI TEMP'])
        UL[0].data = UL[0].data[2157:4212,101:2148]
        bias_data = UL[0].data
        bias_UL[-1].append(bias_data)
        mean,std=analysis_bias(bias_data)
        bias_filtered = filter_bias(bias_data,mean,std,5.)  
        filtered_bias_UL[-1].append(bias_filtered)
        mean_filtered,std_filtered=analysis_bias(bias_filtered)
        mean_bias_UL[-1].append(mean_filtered)
        std_bias_UL[-1].append(std_filtered)
    mean_bias_UL_night_mor.append(np.mean(mean_bias_UL[-1]))        
    mean_std_UL_night_mor.append(np.mean(std_bias_UL[-1]))  
    mean_temp_mor.append(np.mean(temp[-1]))
    for j in range(len(bias_UL[-1])):
        os.chdir(data_dir_base+date[i])
        print(int((j+1)/len(bias_UL[-1])*100),'%')
        for k in range(j+1):
                if j!=k:
                    a = np.array(bias_UL[-1][j],dtype=np.int16)
                    b = np.array(bias_UL[-1][k],dtype=np.int16)
                    diff_bias = np.abs(np.subtract(a,b))
                    mean,std=analysis_bias(diff_bias)
                    diff_bias_filtered = filter_bias(diff_bias,mean,std,5.)
                    rnoise_UL[-1].append(np.std(diff_bias_filtered)/math.sqrt(2.))
                
    mean_rnoise_UL_mor.append(np.mean(rnoise_UL[-1]))     
    std_rnoise_UL_mor.append(np.std(rnoise_UL[-1])) 

bias_UR=[]
mean_bias_UR=[]
std_bias_UR=[]
outliers_UR=[]
filtered_bias_UR=[]
mean_bias_UR_night_mor=[]
mean_std_UR_night_mor=[]
temp=[]
mean_temp_mor=[]
rnoise_UR=[]
mean_rnoise_UR_mor=[]
std_rnoise_UR_mor=[]
mean_rnoise_UR_mor_night=[]
std_rnoise_UR_mor_night=[]


for i in range(len(date_mor)):
    print('Date analyzed now:',date_mor[i])
    os.chdir(data_dir_base+date_mor[i])

#Lower right quadrant
    mean_bias_UR.append([])
    std_bias_UR.append([])
    outliers_UR.append([])
    filtered_bias_UR.append([])
    temp.append([])
    rnoise_UR.append([])
    bias_UR.append([])
    for k in range(len(bias_mor[i])):
        print(int((k+1)/len(bias_mor[i])*100),'%')
        UR = fits.open(bias_mor[i][k])
        temp[-1].append(UR[0].header['HIERARCH OGE AMBI TEMP'])
        UR[0].data = UR[0].data[2157:4212,2149:4196]
        bias_data = UR[0].data
        bias_UR[-1].append(bias_data)
        mean,std=analysis_bias(bias_data)
        bias_filtered = filter_bias(bias_data,mean,std,5.)  
        filtered_bias_UR[-1].append(bias_filtered)
        mean_filtered,std_filtered=analysis_bias(bias_filtered)
        mean_bias_UR[-1].append(mean_filtered)
        std_bias_UR[-1].append(std_filtered)
    mean_bias_UR_night_mor.append(np.mean(mean_bias_UR[-1]))        
    mean_std_UR_night_mor.append(np.mean(std_bias_UR[-1]))  
    mean_temp_mor.append(np.mean(temp[-1]))
    for j in range(len(bias_UR[-1])):
        os.chdir(data_dir_base+date[i])
        print(int((j+1)/len(bias_UR[-1])*100),'%')
        for k in range(j+1):
                if j!=k:
                    a = np.array(bias_UR[-1][j],dtype=np.int16)
                    b = np.array(bias_UR[-1][k],dtype=np.int16)
                    diff_bias = np.abs(np.subtract(a,b))
                    mean,std=analysis_bias(diff_bias)
                    diff_bias_filtered = filter_bias(diff_bias,mean,std,5.)
                    rnoise_UR[-1].append(np.std(diff_bias_filtered)/math.sqrt(2.))
                
    mean_rnoise_UR_mor.append(np.mean(rnoise_UR[-1]))     
    std_rnoise_UR_mor.append(np.std(rnoise_UR[-1])) 





Date analyzed now: 2022-02-16
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-05-15
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-06-19
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-09-04
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-09-06
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-10-03
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-10-14
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-11-07
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-11-12
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-11-27
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-02-16
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed now: 2022-05-15
20 %
40 %
60 %
80 %
100 %
20 %
40 %
60 %
80 %
100 %
Date analyzed no

In [153]:
with open(path+'Readout_Noise_mor.csv', 'w+') as file:
        writer = csv.writer(file)
        data = ['Night','UL','UR','LL','LR','Temp']
        writer.writerow(data)



with open(path+'Mean_Bias_mor.csv', 'w+') as file:
        writer = csv.writer(file)
        data = ['Night','UL','UR','LL','LR','Temp']
        writer.writerow(data)





In [154]:
with open(path+'Readout_Noise_mor.csv', 'a+') as file:
    for i in range(len(date_mor)):
        if (len(bias_mor[i])>=3):
            writer = csv.writer(file)
            data = [date[i],round(mean_rnoise_UL_mor[i],3),round(mean_rnoise_UR_mor[i],3),round(mean_rnoise_LL_mor[i],3),round(mean_rnoise_LR_mor[i],3),round(mean_temp_mor[i],2)]
            writer.writerow(data)
        else:
            writer = csv.writer(file)
            data = [date[i],np.nan,np.nan,np.nan,np.nan,np.nan]
            writer.writerow(data)


with open(path+'Mean_Bias_mor.csv', 'a+') as file:
    for i in range(len(date_mor)):
        if (len(bias_mor[i])>=3):
            writer = csv.writer(file)
            data = [date[i],round(mean_bias_UL_night_mor[i],3),round(mean_bias_UR_night_mor[i],3),round(mean_bias_LL_night_mor[i],3),round(mean_bias_LR_night_mor[i],3),round(mean_temp_mor[i],2)]
            writer.writerow(data)
        else:
            writer = csv.writer(file)
            data = [date[i],np.nan,np.nan,np.nan,np.nan,np.nan]
            writer.writerow(data)







In [155]:
with open(path+'Readout_Noise_eve.csv', 'w+') as file:
        writer = csv.writer(file)
        data = ['Night','UL','UR','LL','LR','Temp']
        writer.writerow(data)



with open(path+'Mean_Bias_eve.csv', 'w+') as file:
        writer = csv.writer(file)
        data = ['Night','UL','UR','LL','LR','Temp']
        writer.writerow(data)




In [156]:
with open(path+'Readout_Noise_eve.csv', 'a+') as file:
    for i in range(len(date_eve)):
        if (len(bias_eve[i])>=3):
            writer = csv.writer(file)
            data = [date[i],round(mean_rnoise_UL_eve[i],3),round(mean_rnoise_UR_eve[i],3),round(mean_rnoise_LL_eve[i],3),round(mean_rnoise_LR_eve[i],3),round(mean_temp_eve[i],2)]
            writer.writerow(data)
        else:
            writer = csv.writer(file)
            data = [date[i],np.nan,np.nan,np.nan,np.nan,np.nan]
            writer.writerow(data)

with open(path+'Mean_Bias_eve.csv', 'a+') as file:
    for i in range(len(date_eve)):
        if (len(bias_eve[i])>=3):
            writer = csv.writer(file)
            data = [date[i],round(mean_bias_UL_night_eve[i],3),round(mean_bias_UR_night_eve[i],3),round(mean_bias_LL_night_eve[i],3),round(mean_bias_LR_night_eve[i],3),round(mean_temp_eve[i],2)]
            writer.writerow(data)
        else:
            writer = csv.writer(file)
            data = [date[i],np.nan,np.nan,np.nan,np.nan,np.nan]
            writer.writerow(data)



